In [ ]:
# import sys
# !python -m pip install pyspark==3.2.1



In [ ]:
# !python -m pip install findspark

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:

scala_version = '2.12'
spark_version = '3.2.1'
packages = [ f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}' , 'org.apache.kafka:kafka-clients:3.3.1' ]
spark = SparkSession.builder.master("local").appName("kafka-example").config("spark.jars.packages", ",".join(packages)).getOrCreate()
spark

In [3]:
#create dataframe from Kafka data
topic_name='RandomNumber'
kafka_server= 'localhost:9092'
kafkaDf=spark.read.format("kafka").option("kafka.bootstrap.servers",kafka_server)\
    .option("subscribe",topic_name).option("startingOffsets","earliest").load()

In [4]:
#show data
kafkaDf.toPandas()

,key,value,topic,partition,offset,timestamp,timestampType
0,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,112,2022-11-07 11:23:01.151,0
1,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,113,2022-11-07 11:23:06.167,0
2,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,114,2022-11-07 11:23:11.178,0
3,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,115,2022-11-07 11:23:16.193,0
4,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,116,2022-11-07 11:23:21.195,0
...,...,...,...,...,...,...,...
99,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,211,2022-11-15 11:57:36.494,0
100,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,212,2022-11-15 11:57:41.511,0
101,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,213,2022-11-15 11:57:46.521,0
102,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,214,2022-11-15 11:57:51.536,0


In [5]:
len(kafkaDf.toPandas())

105

In [6]:
from dis import dis
from time import sleep
from traceback import print_tb
from IPython.display import display,clear_output

In [7]:
#show streaming data using for loop
batchDF=kafkaDf.select(col('topic'),col('offset'),col('value').cast('string').substr(12,1).alias('rand_number'))

In [8]:
display(batchDF.toPandas())

,topic,offset,rand_number
0,RandomNumber,112,0
1,RandomNumber,113,1
2,RandomNumber,114,2
3,RandomNumber,115,3
4,RandomNumber,116,4
...,...,...,...
102,RandomNumber,214,4
103,RandomNumber,215,5
104,RandomNumber,216,5
105,RandomNumber,217,5


In [9]:

for x in range(0,2000):
    try:
        print("Showing live view refreshed every 5 seconds")
        print(f"Sencond passed: {x*5}")
        display(batchDF.toPandas())
        sleep(5)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
        break
print("Live view ends ...")

Showing live view refreshed every 5 seconds
Sencond passed: 80


,topic,offset,rand_number
0,RandomNumber,112,0
1,RandomNumber,113,1
2,RandomNumber,114,2
3,RandomNumber,115,3
4,RandomNumber,116,4
...,...,...,...
120,RandomNumber,232,6
121,RandomNumber,233,6
122,RandomNumber,234,6
123,RandomNumber,235,7


break
Live view ends ...


Perform some data aggreation and show live results

In [ ]:
batchCountDF = batchDF.groupBy('rand_number').count()
for x in range (0,2000):
    try:
        print('showing live view refreshed every 5 seconds')
        print(f"Second passed: {x*5}")
        display(batchCountDF.toPandas())
        sleep(5)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
        break
print("Live view ended")

showing live view refreshed every 5 seconds
Second passed: 0


,rand_number,count
0,7,1
1,3,1
2,8,1
3,0,1
4,5,1
5,6,1
6,9,1
7,1,1
8,4,1
9,2,1


break
Live view ended


Create Streaming dataframe from Kafka

In [18]:
#startingOffset: earliest for batch  and lastest for streaming
streamRawDf = spark.readStream.format("kafka").option("kafka.bootstrap.servers", kafka_server).option("subscribe", topic_name).option("startingOffsets","latest").load()
streamDF = streamRawDf.select(col('topic'),col('offset'),col('value').cast('string').substr(12,1).alias('rand_number'))
checkEvenDF = streamDF.withColumn('Is_Even',col('rand_number').cast('int') % 2 == 0 )

Write stream

In [11]:
from random import randint
randNum=str(randint(0,10000))
q1name = "queryNumber"+randNum
q2name = "queryCheckEven"+randNum

print(q1name+q2name)

queryNumber6927queryCheckEven6927


In [12]:
stream_writer1 = (streamDF.writeStream.queryName(q1name).trigger(processingTime="5 seconds").outputMode("append").format("memory"))
stream_writer2 = (checkEvenDF.writeStream.queryName(q2name).trigger(processingTime="5 seconds").outputMode("append").format("memory"))

In [13]:
query1 = stream_writer1.start()
query1.awaitTermination(1)

False

In [14]:
query2 = stream_writer2.start()

query2.awaitTermination(1)

False

In [15]:
print(query1.name+" "+query2.name)

queryNumber6927 queryCheckEven6927


In [16]:
spark.sql(f"select * from {q1name}").show()

+------------+------+-----------+
|       topic|offset|rand_number|
+------------+------+-----------+
|RandomNumber|   242|          7|
+------------+------+-----------+



View streaming result

In [17]:
result1=spark.sql(f"SELECT * from {query1.name}")
result1.toPandas()

,topic,offset,rand_number
0,RandomNumber,242,7
1,RandomNumber,243,7


In [31]:
# query1.

'fb162849-5302-46a6-a0af-486511d5c64b'

In [19]:
for x in range(0,2000):
    try:
        print("Showing live view refreshed every 5 seconds")
        print(f"second passed: {x*5}")
        result1=spark.sql(f"SELECT * from {query1.name}")
        result2=spark.sql(f"SELECT * from {query2.name}")
        display(result1.toPandas())
        display(result2.toPandas())
        sleep(5)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
        break
print("Live view end")


Showing live view refreshed every 5 seconds
second passed: 5


,topic,offset,rand_number
0,RandomNumber,242,7
1,RandomNumber,243,7
2,RandomNumber,244,7
3,RandomNumber,245,8
4,RandomNumber,246,8
5,RandomNumber,247,8


,topic,offset,rand_number,Is_Even
0,RandomNumber,242,7,False
1,RandomNumber,243,7,False
2,RandomNumber,244,7,False
3,RandomNumber,245,8,True
4,RandomNumber,246,8,True
5,RandomNumber,247,8,True


break
Live view end
